In [0]:
from pyspark.sql.functions import col, current_timestamp, to_timestamp, concat, lit, concat_ws

In [0]:
%run ../config/path_variables

In [0]:
dbutils.fs.ls(f"{raw_file_path}")

In [0]:
races_df = spark.read.option("header", "true").csv(f"{raw_file_path}/races.csv")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("year", IntegerType(), True),
                                      StructField("round", IntegerType(), True),
                                      StructField("circuitId", StringType(), True),
                                      StructField("name", StringType(), True),
                                      StructField("date", StringType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("url", StringType(), True)])
races_df = spark.read \
.schema(races_schema) \
.option("header", "true") \
.csv(f"{raw_file_path}/races.csv")


races_df.printSchema()

In [0]:
from pyspark.sql.functions import col, to_timestamp, concat_ws, current_timestamp
column_select_df = races_df.select(col("raceId"), col("year"), col("round"), col("circuitId"), col("name"), col("date"), col("time"))

column_rename_df = column_select_df.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("circuitId", "circuit_id") \
.withColumnRenamed("lat", "latitude") \
.withColumnRenamed("lng", "longitude") \
.withColumnRenamed("alt", "altitude") \
.withColumn("race_timestamp",to_timestamp(concat_ws(" ",col("date"),col("time")), "yyyy-MM-dd HH:mm:ss")) \
.withColumn("ingestion_date", current_timestamp())



In [0]:
column_rename_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.races")


In [0]:
dbutils.notebook.exit("Success")